In [4]:
from deepface import DeepFace

# Specify the image path
result = DeepFace.verify(
  img1_path = "img1.jpg",
  img2_path = "img2.jpg",
)
print(result)


{'verified': True, 'distance': -2.220446049250313e-16, 'threshold': 0.68, 'model': 'VGG-Face', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 226, 'y': 422, 'w': 899, 'h': 899, 'left_eye': (838, 788), 'right_eye': (512, 776)}, 'img2': {'x': 226, 'y': 422, 'w': 899, 'h': 899, 'left_eye': (838, 788), 'right_eye': (512, 776)}}, 'time': 2.12}


In [46]:
import os
def check2(image_name, folder_path="users/"):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        return

    # Loop through all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Verify the image with each file in the folder
        try:
            result = DeepFace.verify(img1_path=image_name, img2_path=file_path)
            similarity = result['verified']
            print(f"Compared with {file_name}: Match: {similarity}")
            if similarity :
                return False
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
    return True

In [36]:
from deepface import DeepFace

# Specify the image path

def generate_embedding(img_path):
    #img_path = "img2.jpg"

    # Extract face embeddings
    embedding_objs = DeepFace.represent(img_path=img_path)

    # Loop through and extract the embedding
    for embedding_obj in embedding_objs:
        embedding = embedding_obj["embedding"]
        
        # Ensure embedding is a list and check its length (for VGG-Face, it should be 4096)
        assert isinstance(embedding, list)
        assert len(embedding) == 4096
    return embedding
    # Print or use the embedding as needed
#print(embedding_obj)
#print(len(embedding))


In [43]:
import cv2
import os
from deepface import DeepFace
import json  # For saving embeddings to a file

def capture_images_and_generate_embeddings(dir_path="users/", num_images=5, embedding_file="embeddings.json"):
    # Ensure the directory exists
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    # Initialize the webcam (0 is the default camera)
    video_capture = cv2.VideoCapture(0)

    # Check if the camera is opened correctly
    if not video_capture.isOpened():
        print("Error: Could not open webcam.")
        return

    captured_images = 0
    embeddings_data = {}

    while captured_images < num_images:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        if not ret:
            print("Error: Failed to capture image.")
            break

        # Display the frame to the user
        cv2.imshow("Capture Images", frame)

        # Wait for the user to press 'c' to capture an image
        key = cv2.waitKey(1) & 0xFF
        if key == ord("c"):  # Press 'c' to capture image
            img_name = f"{dir_path}/image_{captured_images + 1}.jpg"
            cv2.imwrite(img_name, frame)  # Save the captured image
            print(f"Image {captured_images + 1} saved as {img_name}")

            # Generate embeddings for the captured image
            try:
                embedding = generate_embedding(img_name)
                embeddings_data[img_name] = embedding
                print(f"Embedding for {img_name} generated and saved.")
            except Exception as e:
                print(f"Error generating embedding for {img_name}: {e}")
            
            captured_images += 1

        # Press 'q' to quit if needed
        elif key == ord("q"):
            print("Exiting image capture.")
            break

    # Release the video capture object and close all OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()

    # Save embeddings to a file
    if embeddings_data:
        with open(embedding_file, "w") as f:
            json.dump(embeddings_data, f)
        print(f"Embeddings saved to {embedding_file}")

capture_images_and_generate_embeddings()

Image 1 saved as users//image_1.jpg
Embedding for users//image_1.jpg generated and saved.
Image 2 saved as users//image_2.jpg
Embedding for users//image_2.jpg generated and saved.
Image 3 saved as users//image_3.jpg
Embedding for users//image_3.jpg generated and saved.
Image 4 saved as users//image_4.jpg
Embedding for users//image_4.jpg generated and saved.
Image 5 saved as users//image_5.jpg
Embedding for users//image_5.jpg generated and saved.
Embeddings saved to embeddings.json


In [ ]:
def add_user():
    print("enter the name for user")
    user_name=input()
    print("please face the camera")
    
    capture_images_and_generate_embeddings()
add_user()

In [41]:
import json
import numpy as np

def cosine_similarity(vec1, vec2):
    # Convert lists to numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Compute cosine similarity
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

def check(img_name, embedding_file="embeddings.json"):
    # Generate the embedding for the given image
    v = generate_embedding(img_name)

    # Load existing embeddings from the JSON file
    try:
        with open(embedding_file, "r") as f:
            stored_embeddings = json.load(f)
    except FileNotFoundError:
        print(f"Error: Embedding file '{embedding_file}' not found.")
        return True  # If no embeddings exist, it's a new face

    # Check cosine similarity with each stored embedding
    for stored_img, stored_vector in stored_embeddings.items():
        similarity = cosine_similarity(v, stored_vector)
        print(f"Similarity with {stored_img}: {similarity}")
        if similarity >= 0.9:
            return False  # Found a match with similarity >= 0.9

    return True  # No matches found

    #now check the cosine similarity of this vector with all the vector embedding in embedding.json if similarity between all vecotrs< 0.9 then 
    #output true or else false

In [47]:
import cv2
import os
import time

def button_pressed():
    # Define the directory to save the image
    dir_path = "thief"
    
    # Ensure the directory exists
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    # Initialize the webcam (0 is the default camera)
    video_capture = cv2.VideoCapture(0)

    # Check if the camera is opened correctly
    if not video_capture.isOpened():
        print("Error: Could not open webcam.")
        return

    # Wait for 5 seconds to allow the camera to adjust
    print("Waiting for 5 seconds...")
    time.sleep(5)

    # Capture a single frame
    ret, frame = video_capture.read()

    if not ret:
        print("Error: Failed to capture image.")
    else:
        # Define the image file name
        img_name = os.path.join(dir_path, "captured_image.jpg")
        cv2.imwrite(img_name, frame)  # Save the captured image
        print(f"Image saved as {img_name}")

    # Release the video capture object
    video_capture.release()
    cv2.destroyAllWindows()

    #now we need to check cosine similartiy
    flag=check2(img_name)
    if flag:
        print("car is hijacked!!!!!!")
    

# Example usage:
button_pressed()


Waiting for 5 seconds...
Image saved as thief\captured_image.jpg
Compared with image_1.jpg: Match: True
